# Data Engineering Project

#### We need to, first, do the Python ETL, and afterwards, map it into a RELATIONAL DATABASE using PonyORM

A SMB (small-to-medium business) has recently begun to utilise the data they obtain from
their customers. Unfortunately, their business has multiple areas which all have customer
data specific to that area, and this is fragmented within the organisation. E.g Credit Card
data is only stored by the financial systems, employment within HR, etc. There is not a single
cohesive record representing customers. <br>

The SMB is looking to unify these ahead of further data investigation, and to pool all this data together into a central datastore.
The data provided for this assessment is mock data representing a typical customer-facing
business; these involve data such as names, banking credentials, family attributes, etc.
These data files are provided as a mixed modality in a variety of formats (CSV, JSON, XML,
and TXT).<br>

The work herein requires the processing of these data into a homogenous record, aligning
the same customers from different sources together, which are then automatically entered
into a Relational Database System using modern tools & libraries. <br>

You are expected to read and extract data from these various formats, wrangle the data -
solving inconsistencies if present - and bring data together into a singular format. <br>

These unified records are then to be mapped to a relational database
using PonyORM Entities, with all unified records being entered into the database

### Reading all four data files provided for this assignment

##### Importing essential libraries

In [3]:
import pandas as pd
import numpy as np
from lxml import objectify

#### 1. Reading from CSV file, first

In [4]:
df_csv = pd.read_csv("user_data.csv")
df_csv.rename(columns={"First Name": "first_name", "Second Name": "last_name", 
"Age (Years)":"age","Sex":"sex", "Vehicle Make": "vehicle_make","Vehicle Model":"vehicle_model",
"Vehicle Year":"vehicle_year","Vehicle Type": "vehicle_type"},inplace=True)

In [5]:
df_csv.head()

,first_name,last_name,age,sex,vehicle_make,vehicle_model,vehicle_year,vehicle_type
0,Oliver,Brady,68,Male,Mitsubishi,WRX,2003,Sedan
1,Denis,Jackson,35,Male,Toyota,Canyon Regular Cab,2011,Convertible
2,Joshua,Williams,26,Male,Honda,1500 Crew Cab,1999,Sedan
3,Malcolm,Johnson,37,Male,Dodge,Legacy,1996,"Sedan, Wagon"
4,Ashley,Bates,57,Male,Chevrolet,A8,2018,Sedan


In [6]:
df_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   first_name     1000 non-null   object
 1   last_name      1000 non-null   object
 2   age            1000 non-null   int64 
 3   sex            1000 non-null   object
 4   vehicle_make   1000 non-null   object
 5   vehicle_model  1000 non-null   object
 6   vehicle_year   1000 non-null   int64 
 7   vehicle_type   1000 non-null   object
dtypes: int64(2), object(6)
memory usage: 62.6+ KB


##### Reading from JSON file

In [7]:
## As our data in JSON file doesn't need to normalized, we can straightawy use it to convert to df
df_json = pd.read_json("user_data.json")
df_json.rename(columns={"firstName": "first_name", "lastName": "last_name"},inplace=True)

In [8]:
df_json.head()

,first_name,last_name,age,iban,credit_card_number,credit_card_security_code,credit_card_start_date,credit_card_end_date,address_main,address_city,address_postcode,debt
0,Janet,Whittaker,79,GB06TIPX06791401324359,213175641545275,596,12/17,08/20,Studio 6 Robin court,Christopherland,N49 2LB,NaN
1,Kieran,Heath,83,GB28VQOG22478273510511,4153200250709345,248,11/13,08/21,13 Hale light,East Mandy,BA1 4GB,NaN
2,Jane,Turner,29,GB07EALL99866589295356,6011192438703554,610,03/18,04/18,0 Cooper plains,Sharonville,N56 0PY,"{'amount': '4440.19', 'time_period_years': 0}"
3,Arthur,Mistry,27,GB35LSGC87398305883119,502098808533,948,12/18,06/27,312 Parsons junctions,Williamstown,BN5M 3UU,NaN
4,Julian,King,20,GB94ENYC03030578781434,213160758811140,851,06/21,01/25,396 Leon tunnel,South Bethborough,N9J 1TD,NaN


In [9]:
df_json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   first_name                 1000 non-null   object
 1   last_name                  1000 non-null   object
 2   age                        1000 non-null   int64 
 3   iban                       1000 non-null   object
 4   credit_card_number         1000 non-null   int64 
 5   credit_card_security_code  1000 non-null   int64 
 6   credit_card_start_date     1000 non-null   object
 7   credit_card_end_date       1000 non-null   object
 8   address_main               1000 non-null   object
 9   address_city               1000 non-null   object
 10  address_postcode           1000 non-null   object
 11  debt                       148 non-null    object
dtypes: int64(3), object(9)
memory usage: 93.9+ KB


In [10]:
#Filling NaN in debt as 0 in df_json
df_json['debt'] = df_json['debt'].replace(np.nan, 0)
#re-checking the head of df_json
df_json.head()

,first_name,last_name,age,iban,credit_card_number,credit_card_security_code,credit_card_start_date,credit_card_end_date,address_main,address_city,address_postcode,debt
0,Janet,Whittaker,79,GB06TIPX06791401324359,213175641545275,596,12/17,08/20,Studio 6 Robin court,Christopherland,N49 2LB,0
1,Kieran,Heath,83,GB28VQOG22478273510511,4153200250709345,248,11/13,08/21,13 Hale light,East Mandy,BA1 4GB,0
2,Jane,Turner,29,GB07EALL99866589295356,6011192438703554,610,03/18,04/18,0 Cooper plains,Sharonville,N56 0PY,"{'amount': '4440.19', 'time_period_years': 0}"
3,Arthur,Mistry,27,GB35LSGC87398305883119,502098808533,948,12/18,06/27,312 Parsons junctions,Williamstown,BN5M 3UU,0
4,Julian,King,20,GB94ENYC03030578781434,213160758811140,851,06/21,01/25,396 Leon tunnel,South Bethborough,N9J 1TD,0


In [11]:
df_json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   first_name                 1000 non-null   object
 1   last_name                  1000 non-null   object
 2   age                        1000 non-null   int64 
 3   iban                       1000 non-null   object
 4   credit_card_number         1000 non-null   int64 
 5   credit_card_security_code  1000 non-null   int64 
 6   credit_card_start_date     1000 non-null   object
 7   credit_card_end_date       1000 non-null   object
 8   address_main               1000 non-null   object
 9   address_city               1000 non-null   object
 10  address_postcode           1000 non-null   object
 11  debt                       1000 non-null   object
dtypes: int64(3), object(9)
memory usage: 93.9+ KB


##### Reading from text file

In [12]:
with open('user_data.txt') as f:
    for line in f:
        print(line)

"Debra Wood phoned up at the weekend. They mentioned that they might have gotten their security code incorrect on their billing information, numbers in the wrong order or something. They think that it's 592"

"Congratulations on the promotion Howard! Keep up the good work. Enjoy the extra Â£2000 salary, just make sure not to spend it all like last time Hilary. ;) Remember we still work for Hussain-Adams, at least until you win that yacht. See you on Monday"

"Happy Birthday Ms Molly Dobson! We can't believe you're turning 82 today. Make sure to check out our latest offers on Home Appliances, offer ends midnight."

"INTERNAL MEMO (CONFIDENTIAL): Harrison, we've just found a pretty serious error in Mr Miller's file. He shouldn't be on Â£24515 for his annual pension. This needs rectifying immediately or we're in big trouble! I think it's either Â£27334 or Â£25901... just put the higher one in and hope nobody notices."


##### Reading from XML file

In [13]:

df_xml = pd.read_xml('user_data.xml')
df_xml.rename(columns={"firstName": "first_name", "lastName": "last_name"},inplace=True)

In [14]:
df_xml

,first_name,last_name,age,sex,retired,dependants,marital_status,salary,pension,company,commute_distance,address_postcode
0,Hannah,Jones,21,Female,False,2.0,married or civil partner,20603,0,Ward and Sons,6.56,N06 4LG
1,Tracy,Rowley,50,Female,False,1.0,single,39509,0,"Fuller, King and Robinson",11.01,M1 6JD
2,Shane,Thompson,87,Male,True,2.0,single,53134,13409,NaN,0.00,WF84 1EA
3,Michael,Anderson,85,Male,True,2.0,married or civil partner,58524,39479,NaN,0.00,BN1 7TL
4,Clifford,Mason,64,Male,False,1.0,widowed,32510,0,"Wood, Clarke and Wood",9.53,PH60 9ZT
...,...,...,...,...,...,...,...,...,...,...,...,...
995,Victoria,Khan,42,Female,False,1.0,single,64913,0,Khan Ltd,15.53,W4 5UF
996,Katie,Owen,84,Female,False,2.0,single,47371,0,Atkins-Baxter,12.54,EX88 2UT
997,Kieran,Heath,83,Male,True,1.0,single,44730,13803,NaN,0.00,BA1 4GB
998,Lee,Holland,86,Male,True,2.0,married or civil partner,42968,18108,NaN,0.00,S4 5TR


In [15]:
df_xml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   first_name        1000 non-null   object 
 1   last_name         1000 non-null   object 
 2   age               1000 non-null   int64  
 3   sex               1000 non-null   object 
 4   retired           1000 non-null   bool   
 5   dependants        978 non-null    float64
 6   marital_status    1000 non-null   object 
 7   salary            1000 non-null   int64  
 8   pension           1000 non-null   int64  
 9   company           758 non-null    object 
 10  commute_distance  1000 non-null   float64
 11  address_postcode  1000 non-null   object 
dtypes: bool(1), float64(2), int64(3), object(6)
memory usage: 87.0+ KB


In [16]:
#Here, in df_xml, we have two columns with NaN, dependants and company; dependents NaN can be changed to 0. 
#With company name, it would be suitable to change it to "Unknown" than numeric 0
df_xml['dependants'] = df_xml['dependants'].replace(np.nan, 0)
df_xml['company'] = df_xml['company'].replace(np.nan,"Unknown")

In [17]:
#Re-checking df_xml for the alterations made
df_xml.head()

,first_name,last_name,age,sex,retired,dependants,marital_status,salary,pension,company,commute_distance,address_postcode
0,Hannah,Jones,21,Female,False,2.0,married or civil partner,20603,0,Ward and Sons,6.56,N06 4LG
1,Tracy,Rowley,50,Female,False,1.0,single,39509,0,"Fuller, King and Robinson",11.01,M1 6JD
2,Shane,Thompson,87,Male,True,2.0,single,53134,13409,Unknown,0.00,WF84 1EA
3,Michael,Anderson,85,Male,True,2.0,married or civil partner,58524,39479,Unknown,0.00,BN1 7TL
4,Clifford,Mason,64,Male,False,1.0,widowed,32510,0,"Wood, Clarke and Wood",9.53,PH60 9ZT


In [18]:
df_xml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   first_name        1000 non-null   object 
 1   last_name         1000 non-null   object 
 2   age               1000 non-null   int64  
 3   sex               1000 non-null   object 
 4   retired           1000 non-null   bool   
 5   dependants        1000 non-null   float64
 6   marital_status    1000 non-null   object 
 7   salary            1000 non-null   int64  
 8   pension           1000 non-null   int64  
 9   company           1000 non-null   object 
 10  commute_distance  1000 non-null   float64
 11  address_postcode  1000 non-null   object 
dtypes: bool(1), float64(2), int64(3), object(6)
memory usage: 87.0+ KB


### Changing certain information in DATAFRAMES created as per the instruction given in the text file.

###### First, to update security code of user Debra Wood to 592. 
###### Second, to increase Howard salary by 2000
###### Third, to update Molly Dobson age to 82. 
###### Fourth, to update pension figure to maximum of £27334 or £25901 for Mr. Miller

In [19]:
#First:
# changing credit_card_security_code in df_json to 592 for Debra Wood
df_json.loc[(df_json.first_name == 'Debra') & (df_json.last_name == 'Wood'), ['credit_card_security_code']] = 592

In [20]:
#Second:
#increasing Howard salary by 2000
df_xml.loc[df_xml.first_name == "Howard", 'salary'] += 2000

In [21]:
#Third:
# updating age of Molly Dobson to 82
df_xml.loc[(df_xml.first_name == 'Molly') & (df_xml.last_name == 'Dobson'), ['age']] = 82
df_json.loc[(df_json.first_name == 'Molly') & (df_json.last_name == 'Dobson'), ['age']] = 82
df_csv.loc[(df_csv.first_name == 'Molly') & (df_csv.last_name == 'Dobson'), ['age']] = 82

In [22]:
#Fourth:
#updating pension figure to 27334 for Mr. Miller
df_xml.loc[(df_xml.last_name == 'Miller'), ['pension']] = 27334

In [23]:
df_csv.head()

,first_name,last_name,age,sex,vehicle_make,vehicle_model,vehicle_year,vehicle_type
0,Oliver,Brady,68,Male,Mitsubishi,WRX,2003,Sedan
1,Denis,Jackson,35,Male,Toyota,Canyon Regular Cab,2011,Convertible
2,Joshua,Williams,26,Male,Honda,1500 Crew Cab,1999,Sedan
3,Malcolm,Johnson,37,Male,Dodge,Legacy,1996,"Sedan, Wagon"
4,Ashley,Bates,57,Male,Chevrolet,A8,2018,Sedan


In [24]:
df_json.head()

,first_name,last_name,age,iban,credit_card_number,credit_card_security_code,credit_card_start_date,credit_card_end_date,address_main,address_city,address_postcode,debt
0,Janet,Whittaker,79,GB06TIPX06791401324359,213175641545275,596,12/17,08/20,Studio 6 Robin court,Christopherland,N49 2LB,0
1,Kieran,Heath,83,GB28VQOG22478273510511,4153200250709345,248,11/13,08/21,13 Hale light,East Mandy,BA1 4GB,0
2,Jane,Turner,29,GB07EALL99866589295356,6011192438703554,610,03/18,04/18,0 Cooper plains,Sharonville,N56 0PY,"{'amount': '4440.19', 'time_period_years': 0}"
3,Arthur,Mistry,27,GB35LSGC87398305883119,502098808533,948,12/18,06/27,312 Parsons junctions,Williamstown,BN5M 3UU,0
4,Julian,King,20,GB94ENYC03030578781434,213160758811140,851,06/21,01/25,396 Leon tunnel,South Bethborough,N9J 1TD,0


In [25]:
df_xml.head()

,first_name,last_name,age,sex,retired,dependants,marital_status,salary,pension,company,commute_distance,address_postcode
0,Hannah,Jones,21,Female,False,2.0,married or civil partner,20603,0,Ward and Sons,6.56,N06 4LG
1,Tracy,Rowley,50,Female,False,1.0,single,39509,0,"Fuller, King and Robinson",11.01,M1 6JD
2,Shane,Thompson,87,Male,True,2.0,single,53134,13409,Unknown,0.00,WF84 1EA
3,Michael,Anderson,85,Male,True,2.0,married or civil partner,58524,39479,Unknown,0.00,BN1 7TL
4,Clifford,Mason,64,Male,False,1.0,widowed,32510,0,"Wood, Clarke and Wood",9.53,PH60 9ZT


### Merging all three different datframes
1. First, merging ``df_csv`` and ``df_xml`` on the ``first_name``, ``last_name``, ``age``, and ``sex`` as these are the common column in both of them.<br>
2. Thereafter, merging the obtained result with the ``df_json`` on ``first_name``, ``last_name``, ``address_postcode``, and ``age`` columns as ``df_json`` doesn't have sex column in it

In [26]:
result = pd.merge(df_csv, df_xml, on=["first_name","last_name","age","sex"])
result = pd.merge(result,df_json, on=["first_name","last_name","age", "address_postcode"])
result

,first_name,last_name,age,sex,vehicle_make,vehicle_model,vehicle_year,vehicle_type,retired,dependants,...,commute_distance,address_postcode,iban,credit_card_number,credit_card_security_code,credit_card_start_date,credit_card_end_date,address_main,address_city,debt
0,Oliver,Brady,68,Male,Mitsubishi,WRX,2003,Sedan,False,2.0,...,15.18,EC3R 6QB,GB22ZLAW73114167985782,5141290311954834,946,11/20,01/30,Studio 15 Donna lights,Stuarthaven,"{'amount': '17403.80', 'time_period_years': 2}"
1,Denis,Jackson,35,Male,Toyota,Canyon Regular Cab,2011,Convertible,False,3.0,...,18.16,SW54 8ZR,GB34XERJ15638486218742,2246218069251294,666,07/16,10/30,Studio 38r Gordon wells,Cunninghamfort,0
2,Joshua,Williams,26,Male,Honda,1500 Crew Cab,1999,Sedan,False,0.0,...,8.47,S97 8RB,GB76FFNA64547647754971,4017034567048676352,133,05/18,06/20,173 Jones meadow,Hallstad,0
3,Malcolm,Johnson,37,Male,Dodge,Legacy,1996,"Sedan, Wagon",False,5.0,...,10.64,ME80 4BN,GB46ZLCD77668687243311,4603290222882884096,241,06/17,04/21,Flat 92 Dale courts,North Dominic,0
4,Ashley,Bates,57,Male,Chevrolet,A8,2018,Sedan,False,1.0,...,6.33,NN3 1BS,GB62OYNA11401233406947,2405870932540001,271,08/16,03/31,0 Josephine mountain,Aimeeberg,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Jay,Murray,60,Male,MINI,Outback,1997,Pickup,False,1.0,...,12.36,W11 7TG,GB71MISY27787323725360,30297379355890,737,11/21,02/23,539 Clive fords,Pamelaborough,0
996,Leslie,Clarke,42,Male,Toyota,Sienna,2014,SUV,False,2.0,...,11.12,W08 5JW,GB02BIBF38447263634814,180025330447565,335,02/17,10/26,Studio 02 Atkinson ridges,Grantville,5915.44
997,Natasha,Reid,86,Female,Chevrolet,NX,2015,"Convertible, Coupe",True,2.0,...,0.00,N09 2NR,GB88JZXV39357569891425,4761065208836803,774,06/16,05/24,033 Brown parkway,Rachelburgh,0
998,Samantha,Francis,43,Female,Nissan,Silverado 1500 Extended Cab,2010,Sedan,False,1.0,...,16.64,M04 7UB,GB86GUJZ40891176368758,4875092879849769,6363,12/16,11/20,53 Martin camp,Martinton,0


###### We should make a column "full_name" as in database it could be the most suitable column to actually work as a primary key. "credit_card" numbers are unique and logically work as primary key for identifier, however, what if a customer is smart, and dont want liabilities, and thus declining credit card offer. Is it unethical too to segment people with their credit card number?

In [27]:
#Creating a new column "full_name"
result['full_name'] = result['first_name'] + ' ' + result['last_name']
#chaning order of column to bring "full_name" ahead from the end
cols = ['first_name','last_name','full_name']
result = result[cols + [c for c in result.columns if c not in cols]]
result

,first_name,last_name,full_name,age,sex,vehicle_make,vehicle_model,vehicle_year,vehicle_type,retired,...,commute_distance,address_postcode,iban,credit_card_number,credit_card_security_code,credit_card_start_date,credit_card_end_date,address_main,address_city,debt
0,Oliver,Brady,Oliver Brady,68,Male,Mitsubishi,WRX,2003,Sedan,False,...,15.18,EC3R 6QB,GB22ZLAW73114167985782,5141290311954834,946,11/20,01/30,Studio 15 Donna lights,Stuarthaven,"{'amount': '17403.80', 'time_period_years': 2}"
1,Denis,Jackson,Denis Jackson,35,Male,Toyota,Canyon Regular Cab,2011,Convertible,False,...,18.16,SW54 8ZR,GB34XERJ15638486218742,2246218069251294,666,07/16,10/30,Studio 38r Gordon wells,Cunninghamfort,0
2,Joshua,Williams,Joshua Williams,26,Male,Honda,1500 Crew Cab,1999,Sedan,False,...,8.47,S97 8RB,GB76FFNA64547647754971,4017034567048676352,133,05/18,06/20,173 Jones meadow,Hallstad,0
3,Malcolm,Johnson,Malcolm Johnson,37,Male,Dodge,Legacy,1996,"Sedan, Wagon",False,...,10.64,ME80 4BN,GB46ZLCD77668687243311,4603290222882884096,241,06/17,04/21,Flat 92 Dale courts,North Dominic,0
4,Ashley,Bates,Ashley Bates,57,Male,Chevrolet,A8,2018,Sedan,False,...,6.33,NN3 1BS,GB62OYNA11401233406947,2405870932540001,271,08/16,03/31,0 Josephine mountain,Aimeeberg,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Jay,Murray,Jay Murray,60,Male,MINI,Outback,1997,Pickup,False,...,12.36,W11 7TG,GB71MISY27787323725360,30297379355890,737,11/21,02/23,539 Clive fords,Pamelaborough,0
996,Leslie,Clarke,Leslie Clarke,42,Male,Toyota,Sienna,2014,SUV,False,...,11.12,W08 5JW,GB02BIBF38447263634814,180025330447565,335,02/17,10/26,Studio 02 Atkinson ridges,Grantville,5915.44
997,Natasha,Reid,Natasha Reid,86,Female,Chevrolet,NX,2015,"Convertible, Coupe",True,...,0.00,N09 2NR,GB88JZXV39357569891425,4761065208836803,774,06/16,05/24,033 Brown parkway,Rachelburgh,0
998,Samantha,Francis,Samantha Francis,43,Female,Nissan,Silverado 1500 Extended Cab,2010,Sedan,False,...,16.64,M04 7UB,GB86GUJZ40891176368758,4875092879849769,6363,12/16,11/20,53 Martin camp,Martinton,0


In [28]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   first_name                 1000 non-null   object 
 1   last_name                  1000 non-null   object 
 2   full_name                  1000 non-null   object 
 3   age                        1000 non-null   int64  
 4   sex                        1000 non-null   object 
 5   vehicle_make               1000 non-null   object 
 6   vehicle_model              1000 non-null   object 
 7   vehicle_year               1000 non-null   int64  
 8   vehicle_type               1000 non-null   object 
 9   retired                    1000 non-null   bool   
 10  dependants                 1000 non-null   float64
 11  marital_status             1000 non-null   object 
 12  salary                     1000 non-null   int64  
 13  pension                    1000 non-null   int64 

## NOW, Working with PONY ORM

Re-running this section will try to create instance all over again, so not running it again.

In [29]:
from pony.orm import *
from pony import orm

In [30]:
db = Database()

In [ ]:
class Client(db.Entity):
    first_name = Required(str)
    last_name = Required(str)
    full_name = PrimaryKey(str) #As is is primary key, needed to be unique=True
    age = Required(int)
    sex = Optional(str)
    vehicle_make = Required(str)
    vehicle_model = Required(str)
    vehicle_year = Required(int)
    vehicle_type = Optional(str) #This can be done optional
    retired = Required(bool) #As it is just True/False
    dependants = Required(int)
    marital_status = Required(str)
    salary = Required(int)
    pension = Required(int)
    company = Required(str)
    commute_distance = Optional(float)
    address_postcode = Required(str)
    iban = Required(str, unique=True) #As it is a unique indentifier to a client
    credit_card_number = Required(int, unique=True) # This is too an unique identifier
    credit_card_security_code = Required(int, unique=True) #Same for this
    credit_card_start_date  = Required(str)
    credit_card_end_date = Required(str)
    address_main = Optional(str) #As address_postcode is enough for an indentifier
    address_city = Optional(str) #As address_postcode is enough for an identifier 
    debt = Required(str)

In [33]:
#Establishing connection with the PhpMtAdmin on the host "europa.ashley.work"

db.bind(provider='mysql', host='europa.ashley.work', user='student_bh95bi', passwd='iE93F2@8EhM@1zhD&u9M@K', db='student_bh95bi')

In [34]:
db.generate_mapping(create_tables=True)

In [35]:
set_sql_debug(True)

In [36]:
Client.select().show()

GET NEW CONNECTION
SELECT `c`.`first_name`, `c`.`last_name`, `c`.`full_name`, `c`.`age`, `c`.`sex`, `c`.`vehicle_make`, `c`.`vehicle_model`, `c`.`vehicle_year`, `c`.`vehicle_type`, `c`.`retired`, `c`.`dependants`, `c`.`marital_status`, `c`.`salary`, `c`.`pension`, `c`.`company`, `c`.`commute_distance`, `c`.`address_postcode`, `c`.`iban`, `c`.`credit_card_number`, `c`.`credit_card_security_code`, `c`.`credit_card_start_date`, `c`.`credit_card_end_date`, `c`.`address_main`, `c`.`address_city`, `c`.`debt`
FROM `client` `c`

f...|l...|f...|age|sex|v...|v...|v...|v...|r...|d...|m...|s...|p...|c...|c...
----+----+----+---+---+----+----+----+----+----+----+----+----+----+----+----


In [ ]:
with db_session:
    result.to_sql("Client", db.get_connection())
    print db.select("COUNT (*) FROM Client")    
    